In [1]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def build_model(input_shape=(28,28,1), num_classes=10):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def preprocess(x):
    x = x.astype('float32') / 255.0
    x = x.reshape(-1, 28, 28, 1)
    return x

In [17]:
def main():
    (x_train,y_train),(x_test,y_test)=mnist.load_data()
    x_train=preprocess(x_train)
    x_test = preprocess(x_test)
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    model=build_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # data augementation

    datagen=ImageDataGenerator(rotation_range=8,
                               width_shift_range=0.08,
                                height_shift_range=0.08,
                                zoom_range=0.08)
    datagen.fit(x_train)
    batch_size=128
    epochs=12

    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    ]
    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(x_train)//batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks)

    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {test_acc*100:.2f}%")

    # save model
    model.save("cnn_mnist_model.h5")
    print("Saved model to cnn_mnist_model.h5")

if __name__ == "__main__":
    main()

c:\Users\Devarshi\PycharmProjects\PythonProject\.venv1\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/12
468/468 ━━━━━━━━━━━━━━━━━━━━ 27s 51ms/step - accuracy: 0.8956 - loss: 0.3266 - val_accuracy: 0.9847 - val_loss: 0.0416 - learning_rate: 0.0010
Epoch 2/12
  1/468 ━━━━━━━━━━━━━━━━━━━━ 30s 66ms/step - accuracy: 0.9219 - loss: 0.2388

c:\Users\Devarshi\PycharmProjects\PythonProject\.venv1\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


468/468 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9219 - loss: 0.2388 - val_accuracy: 0.9864 - val_loss: 0.0375 - learning_rate: 0.0010
Epoch 3/12
468/468 ━━━━━━━━━━━━━━━━━━━━ 24s 50ms/step - accuracy: 0.9672 - loss: 0.1077 - val_accuracy: 0.9904 - val_loss: 0.0263 - learning_rate: 0.0010
Epoch 4/12
468/468 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9844 - loss: 0.0741 - val_accuracy: 0.9896 - val_loss: 0.0290 - learning_rate: 0.0010
Epoch 5/12
468/468 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step - accuracy: 0.9760 - loss: 0.0800 - val_accuracy: 0.9922 - val_loss: 0.0214 - learning_rate: 0.0010
Epoch 6/12
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9688 - loss: 0.0884 - val_accuracy: 0.9922 - val_loss: 0.0217 - learning_rate: 0.0010
Epoch 7/12
468/468 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - accuracy: 0.9793 - loss: 0.0681 - val_accuracy: 0.9938 - val_loss: 0.0160 - learning_rate: 0.0010
Epoch 8/12
468/468 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9844 - loss: 0.0499 - val_a

Test accuracy: 99.47%
Saved model to cnn_mnist_model.h5
